# 第七章：函数

使用 def 语句定义函数是所有程序的基础。 本章的目标是讲解一些更加高级和不常见的函数定义与使用模式。 涉及到的内容包括默认参数、任意数量参数、强制关键字参数、注解和闭包。 另外，一些高级的控制流和利用回调函数传递数据的技术在这里也会讲解到。

## 7.1 可接收任意数量参数的函数

### 问题

想构造一个可接受任意数量参数的函数。

### 解决方案

为了能让一个函数接收任意数量的位置参数，可以使用一个 * 参数。例如：

In [1]:
def avg(first, *rest):
    return (first + sum(rest)) / (1 + len(rest))

# Sample use
avg(1, 2)  # 1.5
avg(1, 2, 3, 4)  # 2.5

2.5

在这里例子中，rest 是由所有其它位置参数组成的元组。然后我们在代码中把它当成了一个序列来进行后续的计算。

为了接受任意数量的关键字参数，使用一个以 ** 开头的参数。比如：

In [2]:
import html

def make_element(name, value, **attrs):
    keyvals = [' %s="%s"' % item for item in attrs.items()]
    attr_str = ''.join(keyvals)
    element = '<{name}{attrs}>{value}</{name}>'.format(
                name=name,
                attrs=attr_str,
                value=html.escape(value))
    return element

make_element('item', 'Albatross', size='large', quantity=6)

'<item size="large" quantity="6">Albatross</item>'

在这里，attrs 是一个包含所有被传入进来的关键字参数的字典。

如果你还希望某个函数能同时接受任意数量的位置参数和关键字参数，可以同时使用 * 和 **。比如：

In [3]:
def anyargs(*args, **kwargs):
    print(args)  # A tuple
    print(kwargs)  # A dict

使用这个函数时，所有位置参数会被放到args元组中，所有关键字参数会被放到字典kwargs中。

### 讨论

一个 * 参数只能出现在函数定义中最后一个位置参数后面，而 ** 参数只能出现在最后一个参数。 有一点要注意的是，在 * 参数后面仍然可以定义其他参数。

In [4]:
def a(x, *args, y):  # 参看 7.2
    pass

def b(x, *args, y, **kwargs):
    pass

## 7.2 只接收关键字参数的函数

### 问题

你希望函数的某些参数强制使用关键字参数传递。

### 解决方案

将强制关键字参数放到某个 * 参数或者单个 * 后面就能达到这种效果。比如：

In [ ]:
def recv(maxsize, *, block):
    'Receives a message'
    pass

recv(1024, True)  # TypeError
recv(1024, block=True)  # ok

利用这种技术，我们还能在接受任意多个位置参数的函数中指定关键字参数。比如：

In [5]:
def mininum(*values, clip=None):
    m = min(values)
    if clip is not None:
        m = clip if clip > m else m
    return m

mininum(1, 5, 2, -5, 10)  # Returns -5
mininum(1, 5, 2, -5, 10, clip=0)  # Returns 0

0

### 讨论

很多情况下，使用强制关键字参数会比使用位置参数表意更加清晰，程序也更加具有可读性。 例如，考虑下如下一个函数调用：

In [ ]:
msg = recv(1024, False)

如果调用者对 recv 函数并不是很熟悉，那他肯定不明白那个 False 参数到底来干嘛用的。 但是，如果代码变成下面这样子的话就清楚多了：

In [ ]:
msg = recv(1024, block=False)

强制关键字参数在一些更高级场合同样也很有用。 例如，它们可以被用来在使用 \*args 和 **kwargs 参数作为输入的函数中插入参数。9.11小节有一个这样的例子。

## 7.3 给函数参数增加元信息

### 问题

你写好了一个函数，然后想为这个函数的参数增加一些额外的信息，这样的话其他使用者就能清楚的知道这个函数应该怎么使用。

### 解决方案

使用函数参数注解是一个很好的办法，它能提示程序员应该怎样正确使用这个函数。 例如，下面有一个被注解了的函数：

In [6]:
def add(x:int, y:int) -> int:
    return x + y

python解释器不会对这些注解添加任何的语义。它们不会被类型检查，运行时跟没有加注解之前的效果也没有任何差距。参考9.20小节的一个更加高级的例子，演示了如何利用注解来实现多分派(比如重载函数)。

## 7.4 返回多个值的函数

### 问题

你希望构造一个可以返回多个值的函数。

### 解决方案

为了能返回多个值，函数直接 return 一个元组就行了。例如：

In [7]:
def myfun():
    return 1, 2, 3

a, b, c = myfun()
a

1

### 讨论

myfun() 看起来是返回了多个值，实际上是先创建了一个元组然后返回，这个语法就是用逗号来生成一个元组。当调用返回一个元组的函数的时候，通常会将结果赋值给多个变量，其实就是元组解包。返回结果也可以赋值给单个变量，这时候这个值就是函数返回的那个元组本身了：

In [8]:
x = myfun()
x

(1, 2, 3)

## 7.5 定义有默认参数的函数

### 问题

你想定义一个函数或者方法，它的一个或多个参数是可选的并且有一个默认值。

### 解决方案

定义一个有可选参数的函数是非常简单的，直接在函数定义中给参数指定一个默认值，并放到参数列表最后就行了。例如：

In [ ]:
def spam(a, b=42):
    print(a, b)
    
spam(1)  # ok, a=1, b=42
spam(1, 2)  # ok, a=1, b=2

如果默认参数是一个可修改的容器比如一个列表、集合或者字典，可以使用None作为默认值，就像下面这样：

In [9]:
# Using a list as a default value
def spam(a, b=None):
    if b is None:
        b = []
    ...

如果你并不想提供一个默认值，而是想仅仅测试下某个默认参数是不是有传递进来，可以像下面这样写：

In [10]:
_no_value = object()

def spam(a, b=_no_value):
    if b is _no_value:
        print('No b value supplied')
    ...

### 讨论

有几点需要注意的是：首先，默认参数的值仅仅在函数定义的时候赋值一次。试着运行下面这个例子：

In [11]:
x = 42
def spam(a, b=x):
    print(a, b)
spam(1)
x = 23
spam(1)

1 42
1 42


其次，默认参数的值应该是不可变的对象，比如 None、True、False，数字或字符串。不要写这样的代码：

In [ ]:
def spam(a, b=[]):  # NO!
    ...

如果这么做了，当默认值在其它地方被修改后会影响到下次调用这个函数时的默认值。比如：

In [12]:
def spam(a, b=[]):
    return b
x = spam(1)
print(x)
x.append(99)
x.append('Yow!')
print(x)
print(spam(1))

[]
[99, 'Yow!']
[99, 'Yow!']


为避免这种情况发生，最好是将默认值设为 None，然后在函数里面检查它。

在测试 None 时使用 is 操作符是很重要的，也是这种方案的关键点。这样写是错误的：

In [13]:
def spam(a, b=None):
    if not b:  # NO! use 'b is None' instead
        b = []

这么写的问题在于尽管 None 值确实是被当成 False，但是还有其它的对象（比如长度为 0 的字符串、列表、元组、字典）都会被当做 False。因此，上面的代码会误将一些其它输入也当成是没有输入。比如：

In [ ]:
spam(1)  # ok
x = []
spam(1, x)  # Silent error. x value overwritten by default
spam(1, 0)  # Silent error. 0 ignored
spam(1, '')  # Silent error. '' ignored

## 7.6 定义匿名或内联函数

### 问题

你想为 sort() 操作创建一个很短的回调函数，但又不想用 def 去写一个单行函数， 而是希望通过某个快捷方式以内联方式来创建这个函数。

### 解决方案

当一些函数很简单，仅仅只是计算一个表达式的值的时候，就可以使用lambda表达式来代替了。比如：

In [14]:
add = lambda x, y: x + y
add(2, 3)
add('hello', 'world')

'helloworld'

lambda 表达式典型的使用场景是排序或数据 reduce 等：

In [15]:
names = ['David Beazley', 'Brian Jones', 'Raymond Hettinger', 'Ned Batchelder']
sorted(names, key=lambda name: name.split()[-1].lower())

['Ned Batchelder', 'David Beazley', 'Raymond Hettinger', 'Brian Jones']

## 7.7 匿名函数捕获变量值

### 问题

用 lambda 定义了一个匿名函数，并想在定义时捕获到某些变量的值。

### 解决方案

先看下下面代码的效果：

In [16]:
x = 10
a = lambda y: x + y
x = 20
b = lambda y: x + y
print(a(10))  # 不是 20
print(b(10))

30
30


这其中的原因是 lambda 表达式中的 x 是一个自由变量，在运行时绑定值，而不是定义时绑定，这跟函数的默认值参数定义是不同的。如果想让某个匿名函数在定义时就捕获到值，可以将那个参数值定义成默认参数即可，像下面这样：

In [17]:
x = 10
a = lambda y, x=x: x + y
x = 20
b = lambda y, x=x: x + y
print(a(10))
print(b(10))

20
30


## 7.8 减少可调用对象的参数个数

### 问题

你有一个被其他 python 代码使用的 callable 对象，可能是一个回调函数或者是一个处理器， 但是它的参数太多了，导致调用时出错。

### 解决方案

如果需要减少某个函数的参数个数，你可以使用 functools.partial() 。 partial() 函数允许你给一个或多个参数设置固定的值，减少接下来被调用时的参数个数。 为了演示清楚，假设你有下面这样的函数：

In [18]:
def spam(a, b, c, d):
    print(a, b, c, d)

现在我们使用 partial() 函数来固定某些参数值：

In [19]:
from functools import partial
s1 = partial(spam, 1)  # a = 1
s1(2, 3, 4)
s2 = partial(spam, d=42)  # d = 42
s2(1, 2, 3)
s3 = partial(spam, 1, 2, d=42)  # a = 1, b = 2, d = 42
s3(5)

1 2 3 4
1 2 3 42
1 2 5 42


可以看出 partial() 固定某些参数并返回一个新的 callable 对象。这个新的 callable 接受未赋值的参数， 然后跟之前已经赋值过的参数合并起来，最后将所有参数传递给原始函数。

### 讨论

本节要解决的问题是让原本不兼容的代码可以一起工作。举几个例子：

第一个例子是，假设你有一个点的列表来表示 (x,y) 坐标元组。你可以使用下面的函数来计算两点之间的距离：

In [20]:
points = [(1, 2), (3, 4), (5, 6), (7, 8)]

import math
def distance(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    return math.hypot(x2 - x1, y2 - y1)  # hypot 返回欧几里得范数 sqrt(x*x + y*y)

distance(points[0], points[1])

2.8284271247461903

现在假设你想以某个点为基点，根据点和基点之间的距离来排序所有的这些点。 列表的 sort() 方法接受一个关键字参数来自定义排序逻辑， 但是它只能接受一个单个参数的函数(distance() 很明显是不符合条件的)。 现在我们可以通过使用 partial() 来解决这个问题：

In [21]:
pt = (4, 3)
points.sort(key=partial(distance, pt))
points

[(3, 4), (1, 2), (5, 6), (7, 8)]

更进一步，partial() 通常被用来微调其他库函数所使用的回调函数的参数。 例如，下面是一段代码，使用 multiprocessing 来异步计算一个结果值， 然后这个值被传递给一个接受一个 result 值和一个可选 logging 参数的回调函数：

In [ ]:
def output_result(result, log=None):
    if log is not None:
        log.debug('Got: %r', result)
        
def add(x, y):
    return x + y

if __name__ == '__main__':
    import logging
    from multiprocessing import Pool
    
    logging.basicConfig(level=logging.DEBUG)
    log = logging.getLogger('test')
    
    p = Pool()
    p.apply_async(add, (3, 4), callback=partial(output_result, log=log))
    p.close()
    p.join()

当给 apply_async() 提供回调函数时，通过使用 partial() 传递额外的 logging 参数。 而 multiprocessing 对这些一无所知——它仅仅只是使用单个值来调用回调函数。

作为一个类似的例子，考虑下编写网络服务器的问题，socketserver 模块让它变得很容易。 下面是个简单的echo服务器：

In [ ]:
from socketserver import StreamRequestHandler, TCPServer

class EchoHandler(StreamRequestHandler):
    def handle(self):
        for line in self.rfile:
            self.wfile.write(b'GOT:' + line)
            
serv = TCPServer(('', 15000), EchoHandler)
serv.serve_forever()

不过，假设你想给EchoHandler增加一个可以接受其他配置选项的 __init__ 方法。比如：

In [ ]:
class EchoHandler(StreamRequestHandler):
    # ack is added keyword-only argument. *args, **kwargs are
    # any normal parameters supplied (which are passed on)
    def __init__(self, *args, ack, **kwargs):
        self.ack = ack
        super().__init__(*args, **kwargs)

    def handle(self):
        for line in self.rfile:
            self.wfile.write(self.ack + line)

这么修改后，我们就不需要显式地在TCPServer类中添加前缀了。 但是你再次运行程序后会报类似下面的错误：
```
Exception happened during processing of request from ('127.0.0.1', 59834)
Traceback (most recent call last):
...
TypeError: __init__() missing 1 required keyword-only argument: 'ack'
```

初看起来好像很难修正这个错误，除了修改 socketserver 模块源代码或者使用某些奇怪的方法之外。 但是，如果使用 partial() 就能很轻松的解决——给它传递 ack 参数的值来初始化即可，如下：

In [ ]:
from functools import partial
serv = TCPServer(('', 15000), partial(EchoHandler, ack=b'RECEIVED:'))
serv.serve_forever()

很多时候 partial() 能实现的效果，lambda表达式也能实现。比如，之前的几个例子可以使用下面这样的表达式：

In [ ]:
points.sort(key=lambda p: distance(pt, p))
p.apply_async(add, (3, 4), callback=lambda result: output_result(result,log))
serv = TCPServer(('', 15000),
        lambda *args, **kwargs: EchoHandler(*args, ack=b'RECEIVED:', **kwargs))

这样写也能实现同样的效果，不过相比而已会显得比较臃肿，对于阅读代码的人来讲也更加难懂。 这时候使用 partial() 可以更加直观的表达你的意图(给某些参数预先赋值)。

## 7.9 将单方法的类转换为函数

### 问题

你有一个除 __init__() 方法外只定义了一个方法的类。为了简化代码，你想将它转换成一个函数。

### 解决方法

大多数情况下，可以使用**闭包**来将单个方法的类转换成函数。 举个例子，下面示例中的类允许使用者根据某个模板方案来获取到URL链接地址。

In [ ]:
from urllib.request import urlopen

class UrlTemplate:
    def __init__(self, template):
        self.template = template

    def open(self, **kwargs):
        return urlopen(self.template.format_map(kwargs))  # format_map: 类似 str.format(**mapping)
    
yahoo = UrlTemplate('http://finance.yahoo.com/d/quotes.csv?s={names}&f={fields}')
for line in yahoo.open(names='IBM,AAPL,FB', fields='sl1c1v'):
    print(line.decode('utf-8'))

这个类可以被一个更简单的函数来代替：

In [ ]:
def urltemplate(template):
    def opener(**kwargs):
        return urlopen(template.format_map(kwargs))
    return opener

yahoo = urltemplate('http://finance.yahoo.com/d/quotes.csv?s={names}&f={fields}')
for line in yahoo(names='IBM,AAPL,FB', fields='sl1c1v'):
    print(line.decode('utf-8'))

任何时候只要你碰到需要给某个函数增加额外的状态信息的问题，都可以考虑使用闭包。 相比将你的函数转换成一个类而言，闭包通常是一种更加简洁和优雅的方案。

## 7.10 带额外状态信息的回调函数

### 问题

你的代码中需要依赖到回调函数的使用 (比如事件处理器、等待后台任务完成后的回调等)， 并且你还需要让回调函数拥有额外的状态值，以便在它的内部使用到。

### 解决方案

这一小节主要讨论的是那些出现在很多函数库和框架中的回调函数的使用——特别是跟异步处理有关的。 为了演示与测试，我们先定义如下一个需要调用回调函数的函数：

In [22]:
def apply_async(func, args, *, callback):
    # Compute the result
    result = func(*args)
    
    # Invoke the callback with the result
    callback(result)

实际上，这段代码可以做任何更高级的处理，包括线程、进程和定时器，但是这些都不是我们要关心的。 我们仅仅只需要关注回调函数的调用。下面是一个演示怎样使用上述代码的例子：

In [23]:
def print_result(result):
    print('Got:', result)
    
def add(x, y):
    return x + y

apply_async(add, (2, 3), callback=print_result)

Got: 5


注意到 print_result() 函数仅仅只接受一个参数 result 。不能再传入其他信息。 而当你想让回调函数访问其他变量或者特定环境的变量值的时候就会遇到麻烦。

为了让回调函数访问外部信息，一种方法是使用一个绑定方法来代替一个简单函数。 比如，下面这个类会保存一个内部序列号，每次接收到一个 result 的时候序列号加 1：

In [24]:
class ResultHandler:
    def __init__(self):
        self.sequence = 0
    
    def handler(self, result):
        self.sequence += 1
        print('[{}] Got: {}'.format(self.sequence, result))

使用这个类的时候，你先创建一个类的实例，然后用它的 handler() 绑定方法来做为回调函数：

In [26]:
r = ResultHandler()
apply_async(add, (2, 3), callback=r.handler)
apply_async(add, ('hello', 'world'), callback=r.handler)

[1] Got: 5
[2] Got: helloworld


第二种方式，作为类的替代，可以使用一个闭包捕获状态值，例如：

In [27]:
def make_handler():
    sequence = 0
    def handler(result):
        nonlocal sequence
        sequence += 1
        print('[{}] Got: {}'.format(sequence, result))
    return handler

下面是使用闭包方式的一个例子：

In [28]:
handler = make_handler()
apply_async(add, (2, 3), callback=handler)
apply_async(add, ('hello', 'world'), callback=handler)

[1] Got: 5
[2] Got: helloworld


还有一个更高级的用法，可以使用协程来完成同样的事情：

In [29]:
def make_handler():
    sequence = 0
    while True:
        result = yield
        sequence += 1
        print('[{}] Got: {}'.format(sequence, result))

对于协程，需要使用它的 send() 方法作为回调函数，如下所示：

In [33]:
handler = make_handler()
next(handler)  # 使用之前要调用 next()
apply_async(add, (2, 3), callback=handler.send)
apply_async(add, ('hello', 'world'), callback=handler.send)

[1] Got: 5
[2] Got: helloworld


### 讨论

至少有两种主要方式来捕获和保存状态信息，你可以在一个对象实例(通过一个绑定方法)或者在一个闭包中保存它。 两种方式相比，闭包或许是更加轻量级和自然一点，因为它们可以很简单的通过函数来构造。

如果使用闭包，你需要注意对那些可修改变量的操作。在上面的方案中， nonlocal 声明语句用来指示接下来的变量会在回调函数中被修改。如果没有这个声明，代码会报错。

## 7.11 内联函数回调

### 问题

当你编写使用回调函数的代码的时候，担心很多小函数的扩张可能会弄乱程序控制流。 你希望找到某个方法来让代码看上去更像是一个普通的执行序列。

### 解决方案

通过使用生成器和协程可以使得回调函数内联在某个函数中。 为了演示说明，假设你有如下所示的一个执行某种计算任务然后调用一个回调函数的函数(参考7.10小节)：

In [34]:
def apply_async(func, args, *, callback):
    # Compute the result
    result = func(*args)
    
    # Invoke the callback with the result
    callback(result)

接下来让我们看一下下面的代码，它包含了一个 Async 类和一个 inlined_async 装饰器：

In [35]:
from queue import Queue
from functools import wraps

class Async:
    def __init__(self, func, args):
        self.func = func
        self.args = args
        
def inlined_async(func):
    @wraps(func)
    def wrapper(*args):
        f = func(*args)
        result_queue = Queue()
        result_queue.put(None)
        while True:
            result = result_queue.get()
            try:
                a = f.send(result)
                apply_async(a.func, a.args, callback=result_queue.put)
            except StopIteration:
                break
    return wrapper

这两个代码片段允许你使用 yield 语句内联回调步骤。比如：

In [38]:
def add(x, y):
    return x + y

@inlined_async
def test():
    r = yield Async(add, (2, 3))
    print(r)
    r = yield Async(add, ('hello', 'world'))
    print(r)
    for n in range(6):
        r = yield Async(add, (n, n))
        print(r)
    print('Goodbye')

test()

5
helloworld
0
2
4
6
8
10
Goodbye


你会发现，除了那个特别的装饰器和 yield 语句外，其它地方并没有出现任何的回调函数（其实是在后台定义的）。

### 讨论

本小节会实实在在的测试你关于回调函数、生成器和控制流的知识。

首先，在需要使用到回调的代码中，关键点在于当前计算工作会挂起并在将来的某个时候重启(比如异步执行)。 当计算重启时，回调函数被调用来继续处理结果。apply_async() 函数演示了执行回调的实际逻辑， 尽管实际情况中它可能会更加复杂(包括线程、进程、事件处理器等等)。

计算的暂停与重启思路跟生成器函数的执行模型不谋而合。 具体来讲，yield 操作会使一个生成器函数产生一个值并暂停。 接下来调用生成器的 __next__() 或 send() 方法又会让它从暂停处继续执行。

根据这个思路，这一小节的核心就在 inline_async() 装饰器函数中了。 关键点就是，装饰器会逐步遍历生成器函数的所有 yield 语句，每一次一个。 为了这样做，刚开始的时候创建了一个 result 队列并向里面放入一个 None 值。 然后开始一个循环操作，从队列中取出结果值并发送给生成器，它会持续到下一个 yield 语句， 在这里一个 Async 的实例被接受到。然后循环开始检查函数和参数，并开始进行异步计算 apply_async() 。 然而，这个计算有个最诡异部分是它并没有使用一个普通的回调函数，而是用队列的 put() 方法来回调。

这时候，是时候详细解释下到底发生了什么了。主循环立即返回顶部并在队列上执行 get() 操作。 如果数据存在，它一定是 put() 回调存放的结果。如果没有数据，那么先暂停操作并等待结果的到来。 这个具体怎样实现是由 apply_async() 函数来决定的。 如果你不相信会有这么神奇的事情，你可以使用 multiprocessing 库来试一下， 在单独的进程中执行异步计算操作，如下所示：

In [ ]:
if __name__ == '__main__':
    import multiprocessing
    pool = multiprocessing.Pool()
    apply_async = pool.apply_async
    
    # Run the test function
    test()

实际上你会发现这个真的就是这样的，但是要解释清楚具体的控制流得需要点时间了。

将复杂的控制流隐藏到生成器函数背后的例子在标准库和第三方包中都能看到。 比如，在 contextlib 中的 @contextmanager 装饰器使用了一个令人费解的技巧， 通过一个 yield 语句将进入和离开上下文管理器粘合在一起。 另外非常流行的 Twisted 包中也包含了非常类似的内联回调。

## 7.12 访问闭包中定义的变量

### 问题

你想要扩展函数中的某个闭包，允许它能访问和修改函数的内部变量。

### 解决方案

通常来讲，闭包的内部变量对于外界来讲是完全隐藏的。 但是，你可以通过编写访问函数并将其作为函数属性绑定到闭包上来实现这个目的。例如：

In [39]:
def sample():
    n = 0
    # 闭包函数
    def func():
        print('n=', n)
    
    # Accessor methods for n
    def get_n():
        return n
    
    def set_n(value):
        nonlocal n
        n = value
        
    # Attach as function attributes
    func.get_n = get_n
    func.set_n = set_n
    return func

下面是使用的例子：

In [41]:
f = sample()
f()
f.set_n(10)
f()
f.get_n()

n= 0
n= 10


10

### 讨论

为了说明清楚它是如何工作的，有两点需要解释一下。首先，nonlocal 声明可以让我们编写函数来修改内部变量的值。其次，函数属性允许我们用一种很简单的方式将访问方法绑定到闭包函数上，这个跟实例方法很像（尽管并没有定义任何类）。

还可以进一步扩展，让闭包模拟类的实例。你要做的仅仅是复制上面的内部函数到一个字典实例中并返回它即可。例如：

In [42]:
import sys

class ClosureInstance:
    def __init__(self, locals=None):
        if locals is None:
            locals = sys._getframe(1).f_locals
            
        # Update instance dictionary with callables
        self.__dict__.update((key, value) for key, value in locals.items() if callable(value))
        
    # Redirect special methods
    def __len__(self):
        return self.__dict__['__len__']()
    
# Example use
def Stack():
    items = []
    def push(item):
        items.append(item)
        
    def pop():
        return items.pop()
    
    def __len__():
        return len(items)
    
    return ClosureInstance()

下面是一个交互式会话来演示它是如何工作的：

In [43]:
s = Stack()
s

In [44]:
s.push(10)
s.push(20)
s.push('Hello')
len(s)

3

In [45]:
s.pop()

'Hello'

有趣的是，这个代码运行起来会比一个普通的类定义要快很多。你可能会像下面这样测试它跟一个类的性能对比：

In [46]:
class Stack2:
    def __init__(self):
        self.items = []
    
    def push(self, item):
        self.items.append(item)
        
    def pop(self):
        return self.items.pop()
    
    def __len__(self):
        return len(self.items)

如果这样做，你会得到类似如下的结果：

In [47]:
from timeit import timeit
# Test involving closures
s = Stack()
timeit('s.push(1);s.pop()', 'from __main__ import s')

1.2669026000003214

In [48]:
# Test involving a class
s = Stack2()
timeit('s.push(1);s.pop()', 'from __main__ import s')

1.1888845000066794

结果显示，闭包的方案运行起来要快大概8%，大部分原因是因为对实例变量的简化访问， 闭包更快是因为不会涉及到额外的self变量